Natural Lagnguage Processing Challenge
=================================================

In [ ]:
# Load necessary libraries

import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score , mean_absolute_error
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

Checking and Understanding Datesets

In [ ]:
# Load the ordinary dataset
data = pd.read_csv('data.csv')

# Display the first few rows of the dataset
data.head()






In [ ]:
data.keys()

In [ ]:
# the info of the dataset

data.info(0)

In [ ]:
# Checking for missing NAN
data.isnull().sum()

The Section Processing the Text for Machine Understanding

In [ ]:
data.info()

In [ ]:
# importing all the necessary libraries for this project

import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [ ]:
# Loading the training data
#data = pd.read_csv("data.csv")

In [ ]:
# Initializinig lemmatizer and stop words with fallback
lemmatizer = WordNetLemmatizer()
try:
    stop_words = set(stopwords.words('english')) if _have_stopwords else set()
except Exception:
    stop_words = set()
# this code initializes a lemmatizer and a set of stop words,
# with a fallback to an empty set if there's an issue loading the stop words. 



In [ ]:

# The Pro- reprocessing pf function I wast remove the symbols but keep the numbers
def preprocess_text(text):
    if not isinstance(text, str):
        return ""
    # Removes symbols but keep numbers
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    # Removes stopwords & lemmatize
    tokens = [w for w in text.split() if w not in stop_words]
    tokens = [lemmatizer.lemmatize(w) for w in tokens]
    return ' '.join(tokens)

In [ ]:
# Making  sure the required columns exist
required_cols = {'label','title','text'}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Missing columns in data.csv: {missing}")


In [ ]:
# Apply pre-processing ONLY to title and text, as requested
cols_to_clean = ['title', 'text']



In [ ]:
#  apply per column
for c in cols_to_clean:
    data[c] = data[c].fillna("").astype(str).apply(preprocess_text)

In [ ]:
# creating feature column: title + text
data['clean_text'] = (data['title'] + " " + data['text']).str.strip()

In [ ]:
print(data.head(10))

In [ ]:
# --- Step 6: Train/test split
X = data['clean_text']
y = data['label'].astype(int)  # ensure int labels
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [ ]:
# Vectorize (TF-IDF)
vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))  # unigrams+bigrams often help
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)

In [ ]:
# --- Step 8: Train classifier
model = LogisticRegression(max_iter=2000, n_jobs=None)  # bump max_iter for convergence
model.fit(X_train_vec, y_train)

In [ ]:
# --- Step 9: Evaluate
y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=3)
conf_matrix = confusion_matrix(y_test, y_pred)

In [ ]:
from joblib import dump
print("Accuracy:", round(accuracy, 4))
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


dump(model, "logisticregresionl.joblib")
dump(vectorizer, "tfidf_vectorizer.joblib")





In [ ]:
# Train SGDClassifier
# loss="log_loss" → logistic regression
# loss="hinge" → linear SVM

from sklearn.linear_model import SGDClassifier

model = SGDClassifier(
    loss="log_loss",       # logistic regression
    penalty="l2",          # regularization
    max_iter=1000,
    tol=1e-3,
    random_state=42
)
model.fit(X_train_vec, y_train)


# Evaluate

y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=3)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(accuracy, 4))
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


# Save model & vectorizer

dump(model, "sgd_classifier_model.joblib")
dump(vectorizer, "tfidf_vectorizer.joblib")

In [ ]:

# Using  (SVM)


# This is for saving  models for later
from joblib import dump


# Ensuring am only using the required columns i want title and text for context
required_cols = {"label", "title", "text"}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Missing columns in data.csv: {missing}")

# I cleaned only title and text becasue I only focus of context
for c in ["title", "text"]:
    data[c] = data[c].fillna("").astype(str).apply(preprocess_text)

# Combineing the cleaned into one feature column
data["clean_text"] = (data["title"] + " " + data["text"]).str.strip()


# Train Test and split

X = data["clean_text"]
y = data["label"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)


# Vectorize using TF-IDF

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec  = vectorizer.transform(X_test)


# Training using  another classifier LinearSVC (SVM)

model = LinearSVC()  # the model
model.fit(X_train_vec, y_train)


# Evaluating the meodle

y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=3)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(accuracy, 4))
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


# finally saving the model 

dump(model, "svm_model.joblib")
dump(vectorizer, "tfidf_vectorizer.joblib")




In [ ]:

from joblib import load

# Load saved model and vectorizer
model = load("svm_model.joblib")
vectorizer = load("tfidf_vectorizer.joblib")


# Load validation data
val_data = pd.read_csv("validation_data.csv")

# Preprocess title and text columns
for col in ["title", "text"]:
    val_data[col] = val_data[col].fillna("").astype(str).apply(preprocess_text)

# Combine into one input feature
val_data["clean_text"] = (val_data["title"] + " " + val_data["text"]).str.strip()

# Vectorize using loaded vectorizer
X_val_vec = vectorizer.transform(val_data["clean_text"])

# Predict labels (0 or 1)
predictions = model.predict(X_val_vec)

# Replace label column with predicted labels
val_data["label"] = predictions

# Ensure required output format
expected_cols = ["label", "title", "text", "subject", "date"]
if not set(expected_cols).issubset(val_data.columns):
    raise ValueError(f"Missing expected columns: {set(expected_cols) - set(val_data.columns)}")

# Save output with original format
val_data[expected_cols].to_csv("predicted_validation_data.csv", index=False)
print("✅ Predictions saved to predicted_validation_data_NEW.csv")


In [ ]:

# Natural Lagnguage Processing Challenge (Complement Naive Bayes)



from sklearn.naive_bayes import ComplementNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix



# Ensure required columns
required_cols = {"label", "title", "text"}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Missing columns in data.csv: {missing}")

# Clean title and text
for c in ["title", "text"]:
    data[c] = data[c].fillna("").astype(str).apply(preprocess_text)

# Combine into one column
data["clean_text"] = (data["title"] + " " + data["text"]).str.strip()


# Train/Test split

X = data["clean_text"]
y = data["label"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)


# TF-IDF Vectorization

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


# Train with ComplementNB

model = ComplementNB()
model.fit(X_train_vec, y_train)

# Evaluate

y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=3)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(accuracy, 4))
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


# Save model & vectorizer

dump(model, "complement_nb_model.joblib")
dump(vectorizer, "tfidf_vectorizer.joblib")


In [ ]:

#  (Random Forest)



# Ensure required columns exist
required_cols = {"label", "title", "text"}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Missing columns in data.csv: {missing}")

# Clean title and text
for col in ["title", "text"]:
    data[col] = data[col].fillna("").astype(str).apply(preprocess_text)

# Combine into one text column
data["clean_text"] = (data["title"] + " " + data["text"]).str.strip()


# Train/Test split

X = data["clean_text"]
y = data["label"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)




In [ ]:

# (SGDClassifier)


from sklearn.linear_model import SGDClassifier



# Ensure required columns exist
required_cols = {"label", "title", "text"}
missing = required_cols - set(data.columns)
if missing:
    raise ValueError(f"Missing columns in data.csv: {missing}")

# Clean title and text
for col in ["title", "text"]:
    data[col] = data[col].fillna("").astype(str).apply(preprocess_text)

# Combine into one text column
data["clean_text"] = (data["title"] + " " + data["text"]).str.strip()


# Train/Test split

X = data["clean_text"]
y = data["label"].astype(int)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, random_state=42, stratify=y
)


# TF-IDF Vectorization

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1, 2))
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test)


# Train SGDClassifier
# loss="log_loss" → logistic regression
# loss="hinge" → linear SVM

model = SGDClassifier(
    loss="log_loss",       # logistic regression
    penalty="l2",          # regularization
    max_iter=1000,
    tol=1e-3,
    random_state=42
)
model.fit(X_train_vec, y_train)


# Evaluate

y_pred = model.predict(X_test_vec)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred, digits=3)
conf_matrix = confusion_matrix(y_test, y_pred)

print("Accuracy:", round(accuracy, 4))
print("\nClassification Report:\n", report)
print("\nConfusion Matrix:\n", conf_matrix)


# Save model & vectorizer

dump(model, "sgd_classifier_model.joblib")
dump(vectorizer, "tfidf_vectorizer.joblib")


In [ ]:
import matplotlib.pyplot as plt

# Model names and their accuracies
models = ["SGDClassifier", "Naive Bayes", "SVM", "Logistic Regression" ]
accuracies = [0.9817, 0.9415, 0.995, 0.9875  ]

# Create bar plot
plt.figure(figsize=(9, 6))
bars = plt.bar(models, accuracies, color=["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728" ])

# Add accuracy values on top of each bar
for bar, acc in zip(bars, accuracies):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.002,
             f"{acc:.4f}", ha='center', va='bottom', fontsize=10)

# Labels and title
plt.ylim(0.93, 1.0)  # focus on the high accuracy range
plt.ylabel("Accuracy Score")
plt.title("Model Accuracy Comparison")

plt.show()
